In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('Passengers_SQL').getOrCreate()

In [3]:
passengers_df = spark.read.csv('/content/lax_passengers_header.csv', sep=',', header=True, inferSchema=True)

In [4]:
passengers_df.printSchema()

root
 |-- DataExtractDate: string (nullable = true)
 |-- ReportPeriod: string (nullable = true)
 |-- Terminal: string (nullable = true)
 |-- Arrival_Departure: string (nullable = true)
 |-- Domestic_International: string (nullable = true)
 |-- Passenger_Count: integer (nullable = true)



In [5]:
passengers_df.show(10)

+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|     DataExtractDate|        ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|          Arrival|              Domestic|            490|
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|        Departure|              Domestic|            498|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|          Arrival|              Domestic|            753|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|        Departure|              Domestic|            688|
|05/01/2014 12:00:...|01/01/2006 12:00:...|       Terminal 1|          Arrival|              Domestic|         401535|
|05/01/2014 12:00:...|01/01/2006 12:00:...|     

In [6]:
passengers_df.createOrReplaceTempView('Passengers')

In [7]:
spark.sql('SELECT DISTINCT Terminal FROM Passengers').show(20,truncate=False)

+----------------------------------+
|Terminal                          |
+----------------------------------+
|Terminal 6                        |
|Terminal 8                        |
|Imperial Terminal                 |
|Terminal 3                        |
|Terminal 2                        |
|Tom Bradley International Terminal|
|Terminal 4                        |
|Terminal 5                        |
|Terminal 1                        |
|Misc. Terminal                    |
|Terminal 7                        |
+----------------------------------+



In [20]:
Query="""SELECT CONCAT(SUBSTRING(ReportPeriod,0,3),SUBSTRING(ReportPeriod,7,4)) ReportPeriod,SUM(Passenger_Count) Passenger_Count
         FROM Passengers
         WHERE Terminal IN ("Terminal 1","Terminal 2","Terminal 3","Terminal 4",
                             "Terminal 5","Terminal 6","Terminal 7","Terminal 8","Tom Bradley International Terminal")
         GROUP BY CONCAT(SUBSTRING(ReportPeriod,0,3),SUBSTRING(ReportPeriod,7,4))
         HAVING SUM(Passenger_Count)>5000000
         ORDER BY SUBSTRING(ReportPeriod,7,4)
      """
spark.sql(Query).show(100)

+------------+---------------+
|ReportPeriod|Passenger_Count|
+------------+---------------+
|     12/2010|        5045354|
|     07/2010|        5692210|
|     06/2010|        5311142|
|     08/2010|        5617610|
|     08/2011|        5918484|
|     04/2011|        5028056|
|     05/2011|        5476704|
|     10/2011|        5124462|
|     06/2011|        5631602|
|     07/2011|        6071335|
|     12/2011|        5025839|
|     04/2012|        5160973|
|     09/2012|        5053923|
|     06/2012|        5739649|
|     03/2012|        5252277|
|     05/2012|        5412646|
|     10/2012|        5190645|
|     12/2012|        5176037|
|     07/2012|        6190236|
|     08/2012|        6093051|
|     10/2013|        5386378|
|     12/2013|        5606038|
|     09/2013|        5159762|
|     08/2013|        6160839|
|     03/2013|        5385531|
|     07/2013|        6292610|
|     05/2013|        5557314|
|     06/2013|        5947409|
|     04/2013|        5157963|
|     04